In [22]:
from transformers import pipeline,AutoTokenizer, TFCamembertForSequenceClassification
import sentencepiece
import pandas as pd
import spacy
import numpy as np

df = pd.read_csv("../data/avis/df_cleaned_noYC.csv", index_col= 0)
df.head()

,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société,text_total
0,1,3.0,2023-10-20 03:16:24+00:00,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaire Ça reste une mauvaise exp...
1,1,2.0,2023-10-19 13:10:44+00:00,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...
2,5,1.0,2023-10-19 15:41:51+00:00,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",2023-10-19,MaBanqueBNPParibas,"Chère agence , Chère agence ,je dois bous dire..."
3,1,5.0,2023-10-18 16:13:38+00:00,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",2023-10-17,MaBanqueBNPParibas,Mépris ou incompétence ou les 2 RV pris à l'ag...
4,2,4.0,2023-10-17 22:22:10+00:00,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,2023-10-17,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...


In [23]:
df['text_total'] = df.titre_avis + ". " + df.text_avis
df.text_total = df.text_total.fillna("na")
df.text_total = [message.replace("€", " euros") for message in df.text_total]
df = df.drop([col for col in df.columns if "var_" in col], axis = 1)
df = df.drop(['text_avis','titre_avis'],axis = 1)
df.text_total = [message.replace("€", " euros") for message in df.text_total]
df.head()

,etoiles,n_avis,date_avis,date_experience,Société,text_total
0,1,3.0,2023-10-20 03:16:24+00:00,2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaire. Ça reste une mauvaise ex...
1,1,2.0,2023-10-19 13:10:44+00:00,2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...
2,5,1.0,2023-10-19 15:41:51+00:00,2023-10-19,MaBanqueBNPParibas,"Chère agence ,. Chère agence ,je dois bous dir..."
3,1,5.0,2023-10-18 16:13:38+00:00,2023-10-17,MaBanqueBNPParibas,Mépris ou incompétence ou les 2. RV pris à l'a...
4,2,4.0,2023-10-17 22:22:10+00:00,2023-10-17,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...


In [24]:
# créer une version du texte listé par phrases de plus de un mot et sans ponctuation
# Deux colones: une pour les phrases positives, l'autre pour les négatives


# importing spacy model
nlp = spacy.load("fr_core_news_lg")

#df['text_sent_noPunct'] = ""
def create_review_sent_noPunct(df):
    df['text_sent_noPunct'] = ""
    for i, review in enumerate(df.text_total):
        review = nlp(review)
        sentences_without_punctuation = []
        for sent in review.sents:
            tokens_without_punctuation = [token.text for token in sent if not token.is_punct]
            if len(tokens_without_punctuation) >1:   
                new_sentence = " ".join(tokens_without_punctuation)
                sentences_without_punctuation.append(new_sentence)
        df['text_sent_noPunct'].iloc[i] = sentences_without_punctuation

create_review_sent_noPunct(df)
df.head()

/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_27326/4080025467.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_sent_noPunct'].iloc[i] = sentences_without_punctuation


,etoiles,n_avis,date_avis,date_experience,Société,text_total,text_sent_noPunct
0,1,3.0,2023-10-20 03:16:24+00:00,2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaire. Ça reste une mauvaise ex...,"[Orgueil gestionnaire, Ça reste une mauvaise e..."
1,1,2.0,2023-10-19 13:10:44+00:00,2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...,[Mauvaise expérience en agence physique et en ...
2,5,1.0,2023-10-19 15:41:51+00:00,2023-10-19,MaBanqueBNPParibas,"Chère agence ,. Chère agence ,je dois bous dir...","[Chère agence, Chère agence je dois bous dire ..."
3,1,5.0,2023-10-18 16:13:38+00:00,2023-10-17,MaBanqueBNPParibas,Mépris ou incompétence ou les 2. RV pris à l'a...,"[Mépris ou incompétence ou les 2, RV pris à l'..."
4,2,4.0,2023-10-17 22:22:10+00:00,2023-10-17,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...,[Pas d' imprimé prévu pour transferts de compt...


In [30]:
# Splitting sentences according to sentiment (neg/pos) and granularity (sentence/whole review)

# importing  tokenizer and Camembert classifier
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine")
model = TFCamembertForSequenceClassification.from_pretrained("tblard/tf-allocine")
classifier=  pipeline("text-classification", model = model, tokenizer= tokenizer)

def creating_cols_pos_neg_sents(df):
    # creating columns that will contain all negative and positive sentences for each review
    df['text_clean_sentences_neg'] = ""
    df['text_clean_sentences_pos'] = ""
    for i, sentence_list in enumerate(df['text_sent_noPunct']):
        print("review number", i)
        pos_sent, neg_sent = [],[]
        for sentence in sentence_list:
            sentence = sentence[:512] # nombre de mots max pour Camembert
            try:
                temp = classifier(sentence)[0]['label']
            except Exception as e:
                print(f"Erreur lors du traitement du texte {sentence}. Erreur: {e}")
            if temp == "POSITIVE":
               # print(sentence,"is positive")
                pos_sent.append(sentence)
            elif temp == "NEGATIVE":
                #print(sentence,"is negative")
                neg_sent.append(sentence)

        df['text_clean_sentences_neg'].iloc[i] = ". ".join(neg_sent)
        df['text_clean_sentences_pos'].iloc[i] = ". ".join(pos_sent)
    

# running the functions on the 3 sets: df, validation and test
creating_cols_pos_neg_sents(df)

df = df.drop('text_sent_noPunct', axis =1)

df.head(10)

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.


review number 0


/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_27326/2547438987.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_clean_sentences_neg'].iloc[i] = ". ".join(neg_sent)
/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_27326/2547438987.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_clean_sentences_pos'].iloc[i] = ". ".join(pos_sent)


review number 1
review number 2
review number 3
review number 4
review number 5
review number 6
review number 7
review number 8
review number 9
review number 10
review number 11
review number 12
review number 13
review number 14
review number 15
review number 16
review number 17
review number 18
review number 19
review number 20
review number 21
review number 22
review number 23
review number 24
review number 25
review number 26
review number 27
review number 28
review number 29
review number 30
review number 31
review number 32
review number 33
review number 34
review number 35
review number 36
review number 37
review number 38
review number 39
review number 40
review number 41
review number 42
review number 43
review number 44
review number 45
review number 46
review number 47
review number 48
review number 49
review number 50
review number 51
review number 52
review number 53
review number 54
review number 55
review number 56
review number 57
review number 58
review number 59
review

,etoiles,n_avis,date_avis,date_experience,Société,text_total,text_clean_sentences_neg,text_clean_sentences_pos
0,1,3.0,2023-10-20 03:16:24+00:00,2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaire. Ça reste une mauvaise ex...,Orgueil gestionnaire. Ça reste une mauvaise ex...,
1,1,2.0,2023-10-19 13:10:44+00:00,2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...,Mauvaise expérience en agence physique et en l...,Je change alors pour la banque en ligne avec b...
2,5,1.0,2023-10-19 15:41:51+00:00,2023-10-19,MaBanqueBNPParibas,"Chère agence ,. Chère agence ,je dois bous dir...",,Chère agence. Chère agence je dois bous dire q...
3,1,5.0,2023-10-18 16:13:38+00:00,2023-10-17,MaBanqueBNPParibas,Mépris ou incompétence ou les 2. RV pris à l'a...,Mépris ou incompétence ou les 2. RV pris à l' ...,Voire les 2
4,2,4.0,2023-10-17 22:22:10+00:00,2023-10-17,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...,Pas d' imprimé prévu pour transferts de compte...,
5,1,5.0,2023-10-17 19:19:25+00:00,2023-10-17,MaBanqueBNPParibas,Un peu d'amabilité svp. Je me rend à l'agence ...,Un peu d' amabilité svp. Je me rend à l' agenc...,
6,1,1.0,2023-10-16 15:15:54+00:00,2023-10-16,MaBanqueBNPParibas,Une banque SANS AUCUN RESPECT pour ses…. Une b...,Une banque SANS AUCUN RESPECT pour ses. Une ba...,si vous cherchez une banque honnête et soucieu...
7,1,1.0,2023-10-18 17:08:37+00:00,2023-10-18,MaBanqueBNPParibas,Une étoile parce que je ne peux pas mettre zer...,Une étoile parce que je ne peux pas mettre zer...,Tellement contente de quitter cette banque
8,5,2.0,2023-10-20 14:54:57+00:00,2023-10-20,MaBanqueBNPParibas,J'ai été en contact téléphonique avec…. J'ai é...,J' ai été en contact téléphonique avec,J' ai été en contact téléphonique avec une per...
9,1,1.0,2023-10-20 09:58:16+00:00,2023-10-19,MaBanqueBNPParibas,Alors déjà je laisse une étoile car je…. Alors...,Alors déjà je laisse une étoile car je. Alors ...,Ensuite là j’ étais à découvert de 63 euros EX...


In [31]:
df.to_csv("../data/df_gen_clean_sim_sent.csv")

In [32]:
# removing stopwords
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

def stop_word_text_list(text_list):
    liste_filtrée =[]
    round_count = 1
    for review in text_list:
        if round_count % 100 == 0:
            print(f"{round_count} reviews passed of {len(text_list)} - {round(round_count/len(text_list)*100,2)}% done")            
        filtre = []
        try: 
            texte = nlp(review)
            liste_texte = list(texte)
            for word in liste_texte:
                if str(word) not in stop_words:
                    filtre.append(str(word))
            filt_review = (" ".join(filtre))
        except:
            filt_review = ""
        liste_filtrée.append(filt_review)
        round_count = round_count + 1
    return(liste_filtrée)

In [33]:
# df set
df["text_stop"] = stop_word_text_list(df['text_total'])
df["text_clean_sentences_neg_stop"] = stop_word_text_list(df['text_clean_sentences_neg'])
df["text_clean_sentences_pos_stop"] = stop_word_text_list(df['text_clean_sentences_pos'])
 

100 reviews passed of 105151 - 0.1% done
200 reviews passed of 105151 - 0.19% done
300 reviews passed of 105151 - 0.29% done
400 reviews passed of 105151 - 0.38% done
500 reviews passed of 105151 - 0.48% done
600 reviews passed of 105151 - 0.57% done
700 reviews passed of 105151 - 0.67% done
800 reviews passed of 105151 - 0.76% done
900 reviews passed of 105151 - 0.86% done
1000 reviews passed of 105151 - 0.95% done
1100 reviews passed of 105151 - 1.05% done
1200 reviews passed of 105151 - 1.14% done
1300 reviews passed of 105151 - 1.24% done
1400 reviews passed of 105151 - 1.33% done
1500 reviews passed of 105151 - 1.43% done
1600 reviews passed of 105151 - 1.52% done
1700 reviews passed of 105151 - 1.62% done
1800 reviews passed of 105151 - 1.71% done
1900 reviews passed of 105151 - 1.81% done
2000 reviews passed of 105151 - 1.9% done
2100 reviews passed of 105151 - 2.0% done
2200 reviews passed of 105151 - 2.09% done
2300 reviews passed of 105151 - 2.19% done
2400 reviews passed of 

In [47]:
# saving the dataframe as csv
df.to_csv('../data/avis/general_df_clean_for_sementic_proximity.csv', quoting=csv.QUOTE_NONNUMERIC, index=False)

In [38]:
df2 = df.iloc[:,:]

In [39]:
# Example DataFrame

# Manually quote text columns if they contain commas
text_columns = ['Société','text_total','text_clean_sentences_neg',"text_clean_sentences_pos",'text_stop','text_clean_sentences_neg_stop','text_clean_sentences_pos_stop']  # List of text column names
for column in text_columns:
    # Check if the column value contains a comma and enclose in quotes
    df2[column] = df2[column].apply(lambda x: f'"{x}"' if ',' in x else x)


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105151 entries, 0 to 105150
Data columns (total 11 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   etoiles                        105151 non-null  int64  
 1   n_avis                         105151 non-null  float64
 2   date_avis                      105151 non-null  object 
 3   date_experience                105151 non-null  object 
 4   Société                        105151 non-null  object 
 5   text_total                     105151 non-null  object 
 6   text_clean_sentences_neg       105151 non-null  object 
 7   text_clean_sentences_pos       105151 non-null  object 
 8   text_stop                      105151 non-null  object 
 9   text_clean_sentences_neg_stop  105151 non-null  object 
 10  text_clean_sentences_pos_stop  105151 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 9.6+ MB


In [41]:
import csv
# Save the DataFrame to CSV
df2.to_csv('../data/test.csv', index=False, quoting=csv.QUOTE_NONE, escapechar='\\')

In [42]:
df2.head()

,etoiles,n_avis,date_avis,date_experience,Société,text_total,text_clean_sentences_neg,text_clean_sentences_pos,text_stop,text_clean_sentences_neg_stop,text_clean_sentences_pos_stop
0,1,3.0,2023-10-20 03:16:24+00:00,2023-10-19,MaBanqueBNPParibas,"""Orgueil gestionnaire. Ça reste une mauvaise e...",Orgueil gestionnaire. Ça reste une mauvaise ex...,,"""Orgueil gestionnaire . Ça mauvaise expérience...",Orgueil gestionnaire . Ça mauvaise expérience ...,
1,1,2.0,2023-10-19 13:10:44+00:00,2023-10-19,MaBanqueBNPParibas,"""Mauvaise expérience en agence physique et en ...",Mauvaise expérience en agence physique et en l...,Je change alors pour la banque en ligne avec b...,"""Mauvaise expérience agence physique ligne . M...",Mauvaise expérience agence physique ligne . Ma...,Je change banque ligne beaucoup d ' espoir dig...
2,5,1.0,2023-10-19 15:41:51+00:00,2023-10-19,MaBanqueBNPParibas,"""Chère agence ,. Chère agence ,je dois bous di...",,Chère agence. Chère agence je dois bous dire q...,"""Chère agence , . Chère agence , dois bous mar...",,Chère agence . Chère agence dois bous mari fid...
3,1,5.0,2023-10-18 16:13:38+00:00,2023-10-17,MaBanqueBNPParibas,"""Mépris ou incompétence ou les 2. RV pris à l'...",Mépris ou incompétence ou les 2. RV pris à l' ...,Voire les 2,"""Mépris incompétence 2 . RV pris agence Touque...",Mépris incompétence 2 . RV pris l ' agence Tou...,Voire 2
4,2,4.0,2023-10-17 22:22:10+00:00,2023-10-17,MaBanqueBNPParibas,"""Pas d'imprimé prévu pour transferts de compte...",Pas d' imprimé prévu pour transferts de compte...,,"""Pas imprimé prévu transferts compte compte . ...",Pas d ' imprimé prévu transferts compte compte...,


In [46]:
df.to_csv('../data/test.csv', quoting=csv.QUOTE_NONNUMERIC, index=False)
